In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [6]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [7]:
df['loc']

1     (17749 Collins Ave APT 1002, Sunny Isles Beach...
2     (9705 Collins Ave #601n, Bal Harbour, FL 33154...
3     (8701 Collins Ave #401, Miami Beach, FL 33154,...
4     (3801 Collins Ave #1606, Miami Beach, FL 33140...
5     (8701 Collins Ave #805, Miami Beach, FL 33154,...
6     (4701 Meridian Ave #424, Miami Beach, FL 33140...
7     (300 South Pointe Dr APT 4306, Miami Beach, FL...
8     (18975 Collins Ave #3403, Sunny Isles Beach, F...
9     (16901 Collins Ave Apt 1203, Sunny Isles Beach...
10    (17875 Collins Ave #3806, Sunny Isles Beach, F...
Name: loc, dtype: object

In [8]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [9]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Mansions at Acqualina\r17749 Collins Ave 1002\...,11/10/2021,4/25/2022,110,"$6,250,000.00",4609,"$1,356.04",Marla Cohen,Acqualina Realty,David Vazquez,Brown Harris Stevens,Mansions at Acqualina 17749 Collins Ave 1002 S...,Mansions at Acqualina,17749 Collins Ave 1002 Sunny Isles Beach,"(17749 Collins Ave APT 1002, Sunny Isles Beach...","(25.9400518, -80.12056079999999, 0.0)",25.940052,-80.120561,0.0
2,St Regis\r9705 Collins Ave 601N\rBal Harbour,9/2/2021,4/29/2022,153,"$6,100,000.00",3524,"$1,730.99",Bruno Junqueira,Coldwell Banker Realy,Julian Johnston,The Corcoran Group,St Regis 9705 Collins Ave 601N Bal Harbour,St Regis,9705 Collins Ave 601N Bal Harbour,"(9705 Collins Ave #601n, Bal Harbour, FL 33154...","(25.8887716, -80.1229685, 0.0)",25.888772,-80.122968,0.0
3,Eighty Seven Park\r8701 Collins Ave 401\rMiami...,12/16/2021,4/26/2022,97,"$5,500,000.00",2233,"$2,463.05",Ivan Chorney,"Compass Florida, LLC",Daniel Tzinker,"The Agency Florida, LLC",Eighty Seven Park 8701 Collins Ave 401 Miami B...,Eighty Seven Park,8701 Collins Ave 401 Miami Beach,"(8701 Collins Ave #401, Miami Beach, FL 33154,...","(25.8720993, -80.1218779, 0.0)",25.872099,-80.121878,0.0
4,Mosaic on Miami Beach\r3801 Collins Ave 1606\r...,2/21/2022,4/26/2022,5,"$4,000,000.00",1820,"$2,197.80",Ida Schwartz,"Compass Florida, LLC",Ida Schwartz,"Compass Florida, LLC",Mosaic on Miami Beach 3801 Collins Ave 1606 Mi...,Mosaic on Miami Beach,3801 Collins Ave 1606 Miami Beach,"(3801 Collins Ave #1606, Miami Beach, FL 33140...","(25.8113373, -80.12262849999999, 0.0)",25.811337,-80.122628,0.0
5,Eighty Seven Park\r8701 Collins Ave 805\rMiami...,1/3/2022,4/26/2022,54,"$4,000,000.00",3087,"$2,418.38",Diana Garchitorena,Douglas Elliman,Lianne Graubart,One Sotheby's Intl Realty,Eighty Seven Park 8701 Collins Ave 805 Miami B...,Eighty Seven Park,8701 Collins Ave 805 Miami Beach,"(8701 Collins Ave #805, Miami Beach, FL 33154,...","(25.8720993, -80.1218779, 0.0)",25.872099,-80.121878,0.0
6,The Ritz Carlton\r4701 Meridian Ave 424\rMiami...,7/12/2021,4/25/2022,239,"$3,975,000.00",3440,"$1,239.48",Tristan Alexander,"Compass Florida, LLC",Ilana Levitt,"Compass Florida, LLC",The Ritz Carlton 4701 Meridian Ave 424 Miami B...,The Ritz Carlton,4701 Meridian Ave 424 Miami Beach,"(4701 Meridian Ave #424, Miami Beach, FL 33140...","(25.8227618, -80.13216160000002, 0.0)",25.822762,-80.132162,0.0
7,Portofino Tower\r300 S Pointe Dr 4306\rMiami B...,3/15/2022,4/26/2022,28,"$3,600,000.00",2040,"$1,764.71",John Lennon,South Pointe Drive Realty Inc,Fabrizio Passatore,The Keyes Company,Portofino Tower 300 S Pointe Dr 4306 Miami Beach,Portofino Tower,300 S Pointe Dr 4306 Miami Beach,"(300 South Pointe Dr APT 4306, Miami Beach, FL...","(25.7677332, -80.1349941, 0.0)",25.767733,-80.134994,0.0
8,18975 Collins Condo\r18975 Collins Ave 3403\rS...,1/28/2022,4/26/2022,47,"$3,500,000.00",2271,"$1,541.17",Roland Ortiz,One Sotheby's Intl Realty,Jorge Martinez,One Sotheby's Intl Realty,18975 Collins Condo 18975 Collins Ave 3403 Sun...,,18975 Collins Condo 18975 Collins Ave 3403 Sun...,"(18975 Collins Ave #3403, Sunny Isles Beach, F...","(25.9529783, -80.1191546, 0.0)",25.952978,-80.119155,0.0
9,Jade Signature\r16901 Collins Ave 1203\rSunny ...,10/22/2021,4/28/2022,165,"$3,450,000.00",2501,"$1,379.45",Christiane Brooks,"Compass Florida, LLC",Alina Sanalati,Platinum Ocean Realty LLC,Jade Signature 16901 Collins Ave 1203 Sunny Is...,Jade Signature,16901 Collins Ave 1203 Sunny Isles Beach,"(16901 Collins Ave Apt 1203, Sunny Isles Beach...","(25.9326974, -80.1211241, 0.0)",25.932697,-80.121124,0.0
10,A0cqualina Ocean Residence\r17875 Collins Ave ...,3/1/2022,4/29/2022,42,"$3,225,000.00",2736,"$1,178.73",Marla Cohen,Acqualina Realty,Cindy Clayman,Douglas Elliman,A0cqualina Ocean Residence 17875 Collins Ave 3...,A,

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [14]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [25]:
df.at[10,'building_name']=('Acqualina Ocean Residence')
df.at[10,'address_only']=('17875 Collins Ave 3806')
df.at[8,'building_name']=('18975 Collins Ave 3403')
df.at[8,'address_only']=('18975 Collins Ave 3403')
df.at[6,'building_name']=('The Ritz-Carlton Residences, Miami Beach')

In [11]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [26]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('April 24th - April 30th')

marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [27]:
m.save('index.html')